# Router Chain Example

This notebook demonstrates how to use LangChain's Router Chain to route queries to different specialized prompts based on the input type. The example implements a system that can:

1. **Analyze Requirements**: Routes to a requirement analyst prompt that converts high-level product requirements into GHERKIN format acceptance criteria
2. **Create Test Plans**: Routes to a tester prompt that develops comprehensive test strategies and plans
3. **Generate Test Automation**: Routes to an automation expert prompt that converts manual test cases into automated tests using Playwright and Robot Framework

## Key Components

- Multiple specialized prompts for different roles
- Router chain to determine which prompt to use
- Example inputs showing the full workflow from requirements to test automation

## Prerequisites

- OpenAI API key
- LangChain library
- Python environment with JupyterLab/Notebook


In [48]:
prompt_requirement_analyst = """You are a requirement analyst who is great in understanding the high level product requirements, finding gaps or contradictory requirements, asking questions to fill in missing gaps, and finally documenting them in acceptance criteria in GHERKIN format in an organized way. You have over 20 years of experience in data processing / data analyst domain.

Here is the high level product feature requirement:
{input}
"""

prompt_tester = """You are an expert exploratory tester who is great in discovering edge cases and bugs. You have over 20 years of experience in data processing (etl) domain. You know how to test frontend with the purpose of not only finding bugs but also usability improvements. Additionally you know how to test backend spark applications. Your main goal is to use product's acceptance criteria and come up with concrete test strategy and test plan for the given project. You leverage your domain knowledge to come up with more customer specific use cases that are not covered in the acceptance criteria. Remember, you are not just translating the acceptance crietria into test cases but also adding more value by coming up with more test cases that are not covered in the acceptance criteria

Here is the high level acceptance criteria for the feature:
{input}
"""

prompt_automation = """You are an expert automation tester who is great in writing automated tests. You have over 20 years of experience in data processing (etl) domain. You know how to write automated tests for frontend using playwright and automate backend api tests using ROBOT. Your main goal is to convert the manual tests into automated tests that are reliable and easy to maintain following test best practices. Automate the tests according to the test pyramid shifting left to api as much as possible with minimal e2e tests. Generate automated tests in different sections.

Here is the test case that you need to automate:
{input}
"""

prompts = [
    {
        "name": "Requirement Analyst",
        "description": "Expert in translating high level product requirements into acceptance criteria in Gherkin format",
        "prompt_template": prompt_requirement_analyst,
    },
    {
        "name": "Tester",
        "description": "Expert in writing test strategy and test plan using the acceptance criteria given for the given project",
        "prompt_template": prompt_tester,
    },
    {
        "name": "Automation",
        "description": "Expert in writing automated tests using playwright and ROBOT",
        "prompt_template": prompt_automation,
    }
]

In [26]:
import subprocess

OPEN_AI_KEY = subprocess.run('bash -l -c "getpassw $OPEN_AI_API_KEY"', shell=True, capture_output=True, text=True)
OPEN_AI_KEY = OPEN_AI_KEY.stdout.strip()
LLM_MODEL = "gpt-4o-mini-2024-07-18"

print("")

In [32]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, api_key=OPEN_AI_KEY, model=LLM_MODEL)


In [49]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

prompt_chains = {}
for p in prompts:
    prompt_template = ChatPromptTemplate.from_template(template=p["prompt_template"])
    llmChain = LLMChain(llm=llm, prompt=prompt_template)
    prompt_chains[p["name"]] = llmChain

prompt_name_and_description = "\n".join([f"{p['name']}: {p['description']}" for p in prompts])


In [50]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import RouterOutputParser, LLMRouterChain
from langchain.chains.router import MultiPromptChain

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a language \
model, the model should be able to route to the correct prompt based \
on the input. You will be provided with the name of the prompts and \
high level purpose of the prompt in its description.\

Prompts:
{prompt_name_and_description}

Response Format/Output:
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string // The name of the prompt that user provided OR DEFAULT if no prompt is found
    "next_inputs": string // The input that user provided
}}}}
```

Input:
{{input}}
"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(prompt_name_and_description=prompt_name_and_description)
router_prompt = PromptTemplate(template=router_template, 
                                 input_variables=["input"],
                                 output_parser=RouterOutputParser()
                                )
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)
chain = MultiPromptChain(router_chain = router_chain, 
                         destination_chains=prompt_chains,
                         default_chain=default_chain, 
                         verbose=True)

In [43]:
output = chain.run("Feature: A ETL tool to additionally support filtering of rows using new advance filter expression")
print(output)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Requirement Analyst: {'input': 'Feature: A ETL tool to additionally support filtering of rows using new advance filter expression'}
> Finished chain.
To effectively analyze the high-level product requirement for the ETL tool that supports filtering of rows using a new advanced filter expression, we need to break down the requirement, identify potential gaps or contradictions, and formulate acceptance criteria in GHERKIN format.

### High-Level Requirement Breakdown
1. **Feature Overview**: The ETL tool should allow users to filter rows based on advanced filter expressions.
2. **User Needs**: Users need the ability to define complex filtering criteria to refine the data being processed.
3. **Technical Considerations**: The tool must handle various data types and support logical operators, comparison operators, and possibly functions within the filter expressions.

### Questions to Fill in Gaps
1. **What types of data sources will the ETL tool support for filtering?**
2. **What specific 

In [44]:
output = chain.run("""
Here are the high level feature acceptance criteria of the project:
```gherkin
Feature: Advanced Row Filtering in ETL Tool

  Scenario: User applies a valid advanced filter expression
    Given the user has access to the ETL tool
    When the user inputs a valid advanced filter expression
    Then the ETL tool should filter the rows based on the expression
    And the output should only include rows that match the filter criteria

  Scenario: User inputs an invalid filter expression
    Given the user has access to the ETL tool
    When the user inputs an invalid filter expression
    Then the ETL tool should display an error message indicating the issue
    And the filtering process should not proceed

  Scenario: User applies a filter expression that results in no data
    Given the user has access to the ETL tool
    When the user inputs a filter expression that matches no rows
    Then the ETL tool should return an empty dataset
    And the user should be notified that no data matches the filter criteria

  Scenario: User saves an advanced filter expression for future use
    Given the user has access to the ETL tool
    When the user creates and saves an advanced filter expression
    Then the filter expression should be stored in the user's profile
    And the user should be able to retrieve and apply the saved expression later

  Scenario: User applies a complex filter expression
    Given the user has access to the ETL tool
    When the user inputs a complex filter expression using multiple logical operators
    Then the ETL tool should correctly interpret and apply the expression
    And the output should reflect the correct filtering based on the complex criteria

  Scenario: Performance of filtering with large datasets
    Given the user has access to the ETL tool
    When the user applies an advanced filter expression on a large dataset
    Then the ETL tool should complete the filtering process within an acceptable time frame
```                 
""")
print(output)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Tester: {'input': "Here are the high level feature acceptance criteria of the project:\n```gherkin\nFeature: Advanced Row Filtering in ETL Tool\n\n  Scenario: User applies a valid advanced filter expression\n    Given the user has access to the ETL tool\n    When the user inputs a valid advanced filter expression\n    Then the ETL tool should filter the rows based on the expression\n    And the output should only include rows that match the filter criteria\n\n  Scenario: User inputs an invalid filter expression\n    Given the user has access to the ETL tool\n    When the user inputs an invalid filter expression\n    Then the ETL tool should display an error message indicating the issue\n    And the filtering process should not proceed\n\n  Scenario: User applies a filter expression that results in no data\n    Given the user has access to the ETL tool\n    When the user inputs a filter expression that matches no rows\n    Then the ETL tool should return an empty dataset\n    And the us

In [51]:
output = chain.run("""
Here are is the test plan we need to automate:
### Test Plan

#### Test Cases Derived from Acceptance Criteria

1. **Valid Filter Expression**:
   - Input: A valid filter expression (e.g., `age > 30`)
   - Expected Result: Rows matching the criteria are returned.

2. **Invalid Filter Expression**:
   - Input: An invalid filter expression (e.g., `age >`)
   - Expected Result: An error message is displayed, and no filtering occurs.

3. **No Data Matches**:
   - Input: A filter expression that matches no rows (e.g., `age < 0`)
   - Expected Result: An empty dataset is returned, and the user is notified.

4. **Save Filter Expression**:
   - Input: A valid filter expression to save.
   - Expected Result: The expression is saved and can be retrieved later.

5. **Complex Filter Expression**:
   - Input: A complex filter expression (e.g., `age > 30 AND (city = 'New York' OR city = 'Los Angeles')`)
   - Expected Result: Correct rows are returned based on the complex criteria.

6. **Performance with Large Datasets**:
   - Input: A valid filter expression on a large dataset (e.g., 1 million rows).
   - Expected Result: The filtering process completes within an acceptable time frame (e.g., < 5 seconds).

#### Additional Test Cases (Not Covered in Acceptance Criteria)

1. **Edge Cases for Filter Expressions**:
   - Input: Filter expressions with special characters (e.g., `name LIKE '%John%'`)
   - Input: Filter expressions with null values (e.g., `age IS NULL`)
   - Input: Filter expressions with extreme values (e.g., `age > 100`)

2. **Usability Testing**:
   - Verify that the filter input field provides suggestions or autocomplete for valid expressions.
   - Check if the error messages are user-friendly and provide guidance on correcting the input.

3. **Security Testing**:
   - Input: SQL injection attempts (e.g., `1; DROP TABLE users;`)
   - Expected Result: The system should sanitize inputs and not execute harmful commands.

4. **Concurrent Users**:
   - Simulate multiple users applying filters simultaneously to check for race conditions or performance degradation.

5. **Filter Expression Length**:
   - Input: Extremely long filter expressions (e.g., > 1000 characters).
   - Expected Result: The system should handle long inputs gracefully without crashing.

6. **Data Type Handling**:
   - Input: Filter expressions with mismatched data types (e.g., `age = 'thirty'`)
   - Expected Result: The system should handle type mismatches appropriately and return an error.

7. **Localization and Internationalization**:
   - Test the filtering feature with different locale settings to ensure that it handles various date formats, currency symbols, and language translations correctly.

8. **Undo/Redo Functionality**:
   - Verify if users can undo or redo their filter actions and that the system maintains the correct state.

### Conclusion

This test strategy and plan provide a comprehensive approach to validating the "Advanced Row Filtering" feature in the ETL tool. By not only addressing the acceptance criteria but also considering additional edge cases and usability aspects, we can ensure a robust and user-friendly product.
""")
print(output)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Automation: {'input': 'Here are is the test plan we need to automate:\n### Test Plan\n\n#### Test Cases Derived from Acceptance Criteria\n\n1. **Valid Filter Expression**:\n   - Input: A valid filter expression (e.g., `age > 30`)\n   - Expected Result: Rows matching the criteria are returned.\n\n2. **Invalid Filter Expression**:\n   - Input: An invalid filter expression (e.g., `age >`)\n   - Expected Result: An error message is displayed, and no filtering occurs.\n\n3. **No Data Matches**:\n   - Input: A filter expression that matches no rows (e.g., `age < 0`)\n   - Expected Result: An empty dataset is returned, and the user is notified.\n\n4. **Save Filter Expression**:\n   - Input: A valid filter expression to save.\n   - Expected Result: The expression is saved and can be retrieved later.\n\n5. **Complex Filter Expression**:\n   - Input: A complex filter expression (e.g., `age > 30 AND (city = \'New York\' OR city = \'Los Angeles\')`)\n   - Expected Result: Correct rows are returned